# ETL Consulta Anvisa Produtos Irregulares

Conteudo:

##Pacotes utilizados

In [53]:
import re
import json
import html
import datetime
import requests
import pandas as pd

##Extração dos dados das Consultas
Atualizado: 03/02/2021 - 16/08/2023

- Para o carregamento dos dados foi escolhido a data inicial da criação da plataforma e o dia atual
- Inicialmente foi criado uma seção para depois utilizar os cookies
- E realizado uma requisição para o site, pegando somente o conteudo
- Está requisição vai ser realizada a frente, para verificar o número de paginas contendo as informações, já que o carregamento da página so permite pegar um conjunto de informação de acordo com o número de página
data inicial e final e o número de items(minimo 10,maximo 50)

In [54]:
dataInicial:str = '2021-02-03'
dataFinal:str = '2023-09-05'

In [55]:
session = requests.session()
session.get('https://consultas.anvisa.gov.br/#/dossie/')

<Response [200]>

In [56]:
cookies1 = requests.utils.dict_from_cookiejar(session.cookies)

In [57]:
response = requests.get('https://consultas.anvisa.gov.br/api/dossie/dossie?count=50&filter%5BdataPublicacaoFinal%5D=' + dataFinal +'T03:00:00.000Z&filter%5BdataPublicacaoInicial%5D=' + dataInicial + 'T03:00:00.000Z&filter%5BtipoAssunto%5D=1&page=1',
          headers={
    "accept": "application/json, text/plain, */*",
    "accept-language": "pt-BR,pt;q=0.6",
    "authorization": "Guest",
    "cache-control": "no-cache",
    "if-modified-since": "Mon, 26 Jul 1997 05:00:00 GMT",
    "pragma": "no-cache",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-origin",
    "sec-gpc": "1"
  },
          cookies=cookies1,
verify=False,stream=True).content


/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'consultas.anvisa.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


- O conteudo e decodificado para utf-8 e carregado no formato json

In [58]:
data_json  = json.loads(response.decode('utf-8'))

- Feito isso e realizado o processo de coleta por páginas, usado a informação contida na primeira requisição, que é totalPages

In [59]:
lista_paginas_response:list = []
numero_paginas:int = int(data_json['totalPages'])
for i in range(1,numero_paginas):
  try:
    json_item_response = requests.get('https://consultas.anvisa.gov.br/api/dossie/dossie?count=50&filter%5BdataPublicacaoFinal%5D=' + dataFinal +'T03:00:00.000Z&filter%5BdataPublicacaoInicial%5D=' + dataInicial + 'T03:00:00.000Z&filter%5BtipoAssunto%5D=1&page='+str(i),
            headers={
      "accept": "application/json, text/plain, */*",
      "accept-language": "pt-BR,pt;q=0.6",
      "authorization": "Guest",
      "cache-control": "no-cache",
      "if-modified-since": "Mon, 26 Jul 1997 05:00:00 GMT",
      "pragma": "no-cache",
      "sec-fetch-dest": "empty",
      "sec-fetch-mode": "cors",
      "sec-fetch-site": "same-origin",
      "sec-gpc": "1"
    },
            cookies=cookies1,
    verify=False,stream=True).content
    lista_paginas_response.append(json.loads(json_item_response.decode('utf-8')))
  except requests.HTTPError:
    print("Error de Procura")

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'consultas.anvisa.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'consultas.anvisa.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'consultas.anvisa.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/url

In [60]:
len(lista_paginas_response)

35

- Para está conjunta foi gerado 35 paginas

In [61]:
lista_processo_numero:list[str] = []
for content in range(0, numero_paginas):
  for item in lista_paginas_response:
    lista_processo_numero.append(item['content'][content]['processo']['numero'])

- Dessa forma, queremos somente o número do processo de cada consulta, para gerar um outra requisição

In [62]:
data_json

{'content': [{'idDossie': 42655,
   'totalMedidasCautelares': 1,
   'empresa': {'cnpj': '05823205000190',
    'razaoSocial': 'MAQUIRA INDÚSTRIA DE PRODUTOS ODONTOLÓGICOS S.A.',
    'endereco': '',
    'uf': '',
    'cidade': '',
    'pais': '',
    'cnpjFormatado': '05.823.205/0001-90'},
   'laboratorio': None,
   'processo': {'numero': '25351551558202302',
    'numeroProcessoFormatado': '25351.551558/2023-02'},
   'risco': {'codigo': 21, 'descricao': 'Risco III'},
   'tipoProduto': {'codigo': 8,
    'descricao': 'Produtos para Saúde (Correlatos)'},
   'produtos': [{'registro': '80322400125',
     'produto': 'HYDCAL CIMENTO DE HIDROXIDO DE CALCIO',
     'lotes': '857621, 089522 e 689922.',
     'dataLoteInicial': None,
     'dataFabricacao': None,
     'processoProduto': '25351350119201990'}],
   'dataUltimaMedidaCautelar': '2023-08-31T00:00:00.000-0300',
   'acoesAtividades': ['Recolhimento',
    'Suspensão: Comercialização, Distribuição, Fabricação, Propaganda, Uso']},
  {'idDossie':

In [63]:
listas_data_consulta:list[dict] = []

In [64]:
for item in lista_processo_numero:
  try:
    session = requests.session()
    session.get('https://consultas.anvisa.gov.br/api/dossie/dossie/'+ item)
    item_lista_dados_consulta = json.loads(requests.get(
        'https://consultas.anvisa.gov.br/api/dossie/dossie/'+
        item,
            headers={
                "accept": "application/json, text/plain, */*",
                "accept-language": "pt-BR,pt;q=0.6",
                "authorization": "Guest",
                "cache-control": "no-cache",
                "if-modified-since": "Mon, 26 Jul 1997 05:00:00 GMT",
                "pragma": "no-cache",
                "sec-fetch-dest": "empty",
                "sec-fetch-mode": "cors",
                "sec-fetch-site": "same-origin",
                "sec-gpc": "1"
        },
        cookies=requests.utils.dict_from_cookiejar(session.cookies),
        verify=False).content.decode('UTF-8')
    )
    listas_data_consulta.append(item_lista_dados_consulta)
  except json.decoder.JSONDecodeError:
    print('https://consultas.anvisa.gov.br/api/dossie/dossie/'+item)
    print("Error de Procura")

/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'consultas.anvisa.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'consultas.anvisa.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/urllib3/connectionpool.py:1095: InsecureRequestWarning: Unverified HTTPS request is being made to host 'consultas.anvisa.gov.br'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/url

- Realizado a requisição, capturamos os dados necessario para nossa analise

## Analise e Transformação

- Inicalmente, devemos analisar a forma como os dados estão organizados na lista.
- Sabemos que a lista contêm dados do tipo json.
- É necessario criar um dicionario de dados para facilitar a consulta e saber o tipo de dado retornado

In [65]:
listas_data_consulta[0]

{'idDossie': 42655,
 'totalMedidasCautelares': None,
 'empresa': {'cnpj': '05823205000190',
  'razaoSocial': 'MAQUIRA INDÚSTRIA DE PRODUTOS ODONTOLÓGICOS S.A.',
  'endereco': 'AVENIDA MELVIM JONES, Nº 773',
  'uf': 'PR',
  'cidade': 'MARINGÁ',
  'pais': 'BRASIL',
  'cnpjFormatado': '05.823.205/0001-90'},
 'laboratorio': None,
 'processo': {'numero': '25351551558202302',
  'numeroProcessoFormatado': '25351.551558/2023-02'},
 'risco': {'codigo': 21, 'descricao': 'Risco III'},
 'tipoProduto': {'codigo': 8, 'descricao': 'Produtos para Saúde (Correlatos)'},
 'produtos': [{'registro': '80322400125',
   'produto': 'HYDCAL CIMENTO DE HIDROXIDO DE CALCIO',
   'lotes': '857621, 089522 e 689922.',
   'dataLoteInicial': None,
   'dataFabricacao': None,
   'processoProduto': '25351350119201990'}],
 'dataUltimaMedidaCautelar': None,
 'acoesAtividades': None,
 'resolucaoCBPF': '00000',
 'infracao': 'laudo de análise insatisfatório',
 'dataValidadeCertificado': '2022-12-31T00:00:00.000-0300',
 'situac

|  Empresa | Tipo | Null |
|---------|------|------|
|idDossie| texto | não
|totalMedidasCautelares| texto| sim
|laboratorio| texto| sim
|dataUltimaMedidaCautelar| texto | sim
|acoesAtividades|texto|sim
|resolucaoCBPF|texto|sim
|infracao|texto| não
|dataValidadeCertificado|texto| sim
|situacaoInvestigacao|texto| não
|provaProcessualApensa|texto| não

In [66]:
listas_data_consulta[0]['empresa']

{'cnpj': '05823205000190',
 'razaoSocial': 'MAQUIRA INDÚSTRIA DE PRODUTOS ODONTOLÓGICOS S.A.',
 'endereco': 'AVENIDA MELVIM JONES, Nº 773',
 'uf': 'PR',
 'cidade': 'MARINGÁ',
 'pais': 'BRASIL',
 'cnpjFormatado': '05.823.205/0001-90'}

|  Empresa | Tipo | Null |
|---------|------|------|
|cnpj| texto |
|razaoSocial| texto|
|endereco| texto|
|uf| texto |
|cidade| texto|
|pais| texto|
|cnpjFormatado| texto

In [67]:
listas_data_consulta[0]['processo']

{'numero': '25351551558202302',
 'numeroProcessoFormatado': '25351.551558/2023-02'}

|  Processo | Tipo | Null |
|---------|------|------|
|numero| texto | não|
|numeroProcessoFormatado| texto | não |

In [68]:
listas_data_consulta[0]['risco']

{'codigo': 21, 'descricao': 'Risco III'}

| Risco | Tipo | Null |
|---------|------|------|
|codigo| inteiro | não|
|descricao| texto | não |


In [69]:
listas_data_consulta[0]['tipoProduto']

{'codigo': 8, 'descricao': 'Produtos para Saúde (Correlatos)'}

| topoProduto | Tipo | Null |
|---------|------|------|
|codigo| inteiro | não|
|descricao| texto | não |

In [70]:
listas_data_consulta[0]['produtos']

[{'registro': '80322400125',
  'produto': 'HYDCAL CIMENTO DE HIDROXIDO DE CALCIO',
  'lotes': '857621, 089522 e 689922.',
  'dataLoteInicial': None,
  'dataFabricacao': None,
  'processoProduto': '25351350119201990'}]

| Produtos | Tipo | Null |
|---------|------|------|
|registro| texto | sim|
|produto| texto | não |
|lotes| texto | sim |
|dataLoteInicial| texto | sim |
|dataFabricacao| texto | sim |
|processoProduto| texto | sim |

In [71]:
listas_data_consulta[0]['assunto']

{'codigo': '70446', 'descricao': 'PRODUTOS PARA SAÚDE: Laudo de Análise'}

| Assunto | Tipo | Null |
|---------|------|------|
| codigo  | texto|      |
|descricao| texto|      |

In [72]:
listas_data_consulta[0]['medidasCautelares']

[{'expediente': '0896017238',
  'acoesAtividades': ['Recolhimento',
   'Suspensão: Comercialização, Distribuição, Fabricação, Propaganda, Uso'],
  'situacaoMedidaCautelar': 'Ativa',
  'situacaoDocumento': 'Publicado deferimento',
  'numeroDOU': '167',
  'dataPublicacao': '2023-08-31T00:00:00.000-0300',
  'numeroResolucao': '3.267',
  'dataResolucao': '2023-08-30T00:00:00.000-0300',
  'assunto': {'codigo': '70351',
   'descricao': 'Ações de Fiscalização em Vigilância Sanitária'},
  'motivacao': '<p class="MsoNormal" style="text-align: justify; text-justify: inter-ideograph; margin: 0cm 0cm 8pt;"><span style="font-size: medium;"><span class="ui-provider"><span style="font-family: Calibri;">Considerando os Laudos de An&aacute;lises do tipo amostra &uacute;nica, n&ordm; </span></span><em><span style="font-family: \'Calibri\',sans-serif; color: black; mso-ascii-theme-font: minor-latin; mso-hansi-theme-font: minor-latin; mso-bidi-theme-font: minor-latin;">884.1P.0/2023, 885.1P.0/2023 e 937.1



| Assunto | Tipo | Null |
|---------|------|------|
|acoesAtividades| vetor|
|situacaoMedidaCautelar| texto|
|situacaoDocumento| texto|
|numeroDOU| texto|
|dataPublicacao| texto|
|numeroResolucao| texto|
|dataResolucao| texto|
|assunto| conjunto |
|motivacao| html |

In [73]:
listas_data_consulta[0]['medidasCautelares'][0]['assunto']

{'codigo': '70351',
 'descricao': 'Ações de Fiscalização em Vigilância Sanitária'}

| Assunto | Tipo | Null |
|---------|------|------|
| codigo  | texto|      |
|descricao| texto|      |

- Observando o dicionarios temos alguns campos que são nulos, cadeia de caracteres, conjunto de dados e vetor.
- Dado a extrutura do formato dos dados desta requisição

### Pre-processamento dos Dados da Consulta
Nesta parte será aplicado uma padronização dos dados, e extruturação deles, será repasado OS

- Será criado as colunas A's que armazena as ações, a qual caso tenha menos de 10 items será prenchido pela palavra N/A

In [74]:
to_clean_tags = re.compile('<.*?>')

- Antes de construir o dataset com os dados, devemos remover algum caracteres estranhos dessa forma

In [75]:
consulta:pd.DataFrame = pd.DataFrame()

In [76]:
for item_consulta in listas_data_consulta:
  if 'error' not in item_consulta.keys():
    for item_medida in item_consulta['medidasCautelares']:
      for item_produto in item_consulta['produtos']:
          item_consultas:pd.DataFrame = pd.DataFrame(
            data={
                "empresa_cnpj": [item_consulta['empresa']['cnpj']],
                "empresa_razao_social": [item_consulta['empresa']['razaoSocial']],
                "empresa_endereco": [item_consulta['empresa']['endereco']],
                "empresa_uf": [item_consulta['empresa']['uf']],
                "empresa_cidade": [item_consulta['empresa']['cidade']],
                "empresa_pais": [item_consulta['empresa']['pais']],
                "empresa": [item_consulta['empresa']['cnpjFormatado']],
                "assunto_codigo": [item_consulta['assunto']['codigo']],
                "assunto_descricao": [item_consulta['assunto']['descricao']],
                "processo_numero":[item_consulta['processo']['numero']],
                "processo_numero_formatado": [item_consulta['processo']['numeroProcessoFormatado']],
                "risco_codigo": [item_consulta['risco']['codigo']],
                "risco_descricao": [item_consulta['risco']['descricao']],
                "tipo_produto_codigo": [item_consulta['tipoProduto']['codigo']],
                "tipo_produto_descricao": [item_consulta['tipoProduto']['descricao']],
                "registro":[item_produto['registro']],
                "produto":[item_produto['produto']],
                "lotes":[item_produto['lotes']],
                "data_lote_inicial":[item_produto['dataLoteInicial']],
                "data_fabricacao":[item_produto['dataFabricacao']],
                "processo_produto":[item_produto['processoProduto']],
                "expediente": [item_medida['expediente']],
                "Validado": ["OK"],
                "re":[item_medida['numeroResolucao']],
                "dia": [datetime.date.fromisoformat(item_medida['dataResolucao'][0:10]).day],
                "mês":[datetime.date.fromisoformat(item_medida['dataResolucao'][0:10]).ctime()[4:7]],
                "ano":[datetime.date.fromisoformat(item_medida['dataResolucao'][0:10]).year],
                "data de publicação da medida (site)":[datetime.date.fromisoformat(item_medida['dataPublicacao'][0:10]).strftime("%d/%m/%Y")],
                "categoria": [item_consulta['tipoProduto']['descricao']],
                "Acões": [' '.join(item_medida['acoesAtividades'])],
                "motivação":[re.sub(to_clean_tags,'',html.unescape(item_medida['motivacao']))]
            }
          )
      consulta = pd.concat([consulta, item_consultas], ignore_index=True)

In [77]:
consulta.sort_values(by='data de publicação da medida (site)', ignore_index=True, inplace=True)

In [78]:
consulta

,empresa_cnpj,empresa_razao_social,empresa_endereco,empresa_uf,empresa_cidade,empresa_pais,empresa,assunto_codigo,assunto_descricao,processo_numero,...,expediente,Validado,re,dia,mês,ano,data de publicação da medida (site),categoria,Acões,motivação
0,13343059000124,LW INDUSTRIA E COMERCIO DE PESCADOS LTDA EPP (...,"Av Domingos da Silva Candemil, n. 950, Centro,...",None,None,BRASIL,13.343.059/0001-24,70476,ALIMENTOS - Falsificação,25351043037202297,...,0361646221,OK,291,28,Jan,2022,01/02/2022,Alimento,Proibição: Comercialização (revogado por 42291...,"Considerando o disposto no inciso XV, art. 7º ..."
1,01057428000133,DESCARPACK DESCARTAVEIS DO BRASIL LTDA,AV QUEIROZ FILHO 1700 Edifício Sky Tower - Sal...,SP,SÃO PAULO,BRASIL,01.057.428/0001-33,70451,PRODUTOS PARA SAÚDE: Recolhimento Voluntário,25351012781202169,...,0533004211,OK,893,26,Feb,2021,01/03/2021,Produtos para Saúde (Correlatos),Recolhimento: Voluntário Suspensão: Comerciali...,Considerando a ação de campo de recolhimento i...
2,49351786000261,BAXTER HOSPITALAR LTDA,"AV. ENGº EUSÉBIO STEVAUX, 2555",SP,SÃO PAULO,BRASIL,49.351.786/0002-61,70451,PRODUTOS PARA SAÚDE: Recolhimento Voluntário,25351007106202118,...,0532957214,OK,894,26,Feb,2021,01/03/2021,Produtos para Saúde (Correlatos),Recolhimento: Voluntário Suspensão: Comerciali...,Considerando o Alerta de Tecnovigilância nº 33...
3,28626346000119,DALA COSTA &SILVA LTDA.,"Rua Frei Fabiano de Cristo, 481, Jardim das Am...",None,None,BRASIL,28.626.346/0001-19,70457,COSMÉTICOS: Produto sem Registro/Empresa sem AFE,25351426981202014,...,0737928215,OK,875,26,Feb,2021,01/03/2021,Cosmético,"Apreensão Proibição: Comercialização, Distribu...",Considerando a comercialização e fabricação do...
4,56998701000116,ABBOTT LABORATÓRIOS DO BRASIL LTDA,"RUA MICHIGAN, N° 735",SP,SÃO PAULO,BRASIL,56.998.701/0001-16,70451,PRODUTOS PARA SAÚDE: Recolhimento Voluntário,25351012828202194,...,0543184211,OK,895,26,Feb,2021,01/03/2021,Produtos para diagnóstico de uso in vitro,Recolhimento: Voluntário Suspensão: Comerciali...,Considerando o Alerta de Tecnovigilância n. 33...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1337,03007331000141,EBAZAR.COM.BR. LTDA,"AV DAS NACOES UNIDAS 3000, 3003, PARTE A",SP,OSASCO,BRASIL,03.007.331/0001-41,70819,MEDICAMENTO - INTERNET - Empresa sem AFE / Pro...,25351401614202270,...,4862495221,OK,3.558,27,Oct,2022,31/10/2022,Medicamento,"Apreensão Proibição: Comercialização, Distribu...",Comprovação da propaganda e comercialização do...
1338,08574810000163,SAN JULLY INDÚSTRIA E COMÉRCIO DE COSMÉTICOS LTDA,"AVENIDA SANTO ANTONIO, N° 115",GO,ANÁPOLIS,BRASIL,08.574.810/0001-63,70458,COSMÉTICOS: Produto sem Registro/Empresa com AFE,25351323625202210,...,4601735226,OK,2.837,25,Aug,2022,31/10/2022,Cosmético,"Proibição: Armazenamento, Comercialização, Dis...",Considerando a comercialização/exposição à ven...
1339,Desconhecido,Não Identificada,,None,None,BRASIL,,70819,MEDICAMENTO - INTERNET - Empresa sem AFE / Pro...,25351410772202211,...,4852012228,OK,3.558,27,Oct,2022,31/10/2022,Medicamento,"Apreensão Proibição: Comercialização, Distribu...",\r\nComprovação da propaganda e comercializaçã...
1340,31722743000107,MINOXFARMA LTDA,"R. ARTUR JOSE DA SILVA, 67 - CONJUNTO CRISTINA...",None,None,BRASIL,31.722.743/0001-07,70819,MEDICAMENTO - INTERNET - Empresa sem AFE / Pro...,25351443945202287,...,4858422223,OK,3.558,27,Oct,2022,31/10/2022,Medicamento,"Apreensão Proibição: Comercialização, Distribu...",Comprovação da comercialização dos produtos qu...


## Carregamento dos Dados

In [79]:
consulta.to_excel('consulta'+dataFinal+dataInicial+'.xlsx')

In [81]:
consulta.to_csv('dados_consulta_'+dataFinal+dataInicial+'.csv')